## Imports

In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE  # Import SMOTE
import random
from tqdm import tqdm
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import shuffle

# Loading

In [2]:
DATASET_DIRECTORY = '../../dataset/'  # If your dataset is within your python project directory, change this to the relative path to your dataset
csv_filepaths = [filename for filename in os.listdir(DATASET_DIRECTORY) if filename.endswith('.csv')]

print(csv_filepaths)

# If there are more than X CSV files, randomly select X files from the list
sample_size = 1

if len(csv_filepaths) > sample_size:
    csv_filepaths = random.sample(csv_filepaths, sample_size)
    print(csv_filepaths)

csv_filepaths.sort()

# list of csv files used
data_sets = csv_filepaths

num_cols = [
    'flow_duration', 'Header_Length', 'Duration', 'Rate', 'Srate', 'ack_count', 'syn_count',
    'fin_count', 'urg_count', 'rst_count', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 
    'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight'
]
cat_cols = [
    'Protocol Type', 'Drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
    'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'HTTP', 'HTTPS', 
    'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC'
]

full_data = pd.DataFrame()
for data_set in data_sets:
    print(f"data set {data_set} out of {len(data_sets)} \n")
    data_path = os.path.join(DATASET_DIRECTORY, data_set)
    df = pd.read_csv(data_path)
    full_data = pd.concat([full_data, df])

# prints an instance of each class
print("Before encoding:")
unique_labels = full_data['label'].unique()
for label in unique_labels:
    print(f"First instance of {label}:")
    print(full_data[full_data['label'] == label].iloc[0])

# Shuffle data
full_data = shuffle(full_data, random_state=1)

# prove if the data is loaded properly
print("Real data:")
print(full_data[:2])
print(full_data.shape)

# Assuming 'label' is the column name for the labels in the DataFrame `synth_data`
unique_labels = full_data['label'].nunique()

# Print the number of unique labels
print(f"There are {unique_labels} unique labels in the dataset.")

class_counts = full_data['label'].value_counts()
print(class_counts)

# Display the first few entries to verify the changes
print(full_data)

['part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv']
['part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv']
data set part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv out of 1 

Before encoding:
First instance of DDoS-SYN_Flood:
flow_duration                  0.0
Header_Length                53.46
Protocol Type                 5.94
Duration                     63.36
Rate                        1.1458
Srate                       1.1458
Drate                          0.0
fin_flag_number                0.0
syn_flag_number                1.0
rst_flag_number                0.0
psh_flag_number                0.0
ack_flag_number                0.0
ece_flag_number                0.0
cwr_flag_number                0.0
ack_count           

# Preprocessing
## Encoding Labels

In [3]:
label_encoder = LabelEncoder()
full_data['label'] = label_encoder.fit_transform(full_data['label'])

# Store label mappings
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("Label mappings:", label_mapping)

# Retrieve the numeric codes for classes
class_codes = {label: label_encoder.transform([label])[0] for label in label_encoder.classes_}

# Print specific instances after label encoding
print("After encoding:")
for label, code in class_codes.items():
    # Print the first instance of each class
    print(f"First instance of {label} (code {code}):")
    print(full_data[full_data['label'] == code].iloc[0])


Label mappings: {0: 'Backdoor_Malware', 1: 'BenignTraffic', 2: 'BrowserHijacking', 3: 'CommandInjection', 4: 'DDoS-ACK_Fragmentation', 5: 'DDoS-HTTP_Flood', 6: 'DDoS-ICMP_Flood', 7: 'DDoS-ICMP_Fragmentation', 8: 'DDoS-PSHACK_Flood', 9: 'DDoS-RSTFINFlood', 10: 'DDoS-SYN_Flood', 11: 'DDoS-SlowLoris', 12: 'DDoS-SynonymousIP_Flood', 13: 'DDoS-TCP_Flood', 14: 'DDoS-UDP_Flood', 15: 'DDoS-UDP_Fragmentation', 16: 'DNS_Spoofing', 17: 'DictionaryBruteForce', 18: 'DoS-HTTP_Flood', 19: 'DoS-SYN_Flood', 20: 'DoS-TCP_Flood', 21: 'DoS-UDP_Flood', 22: 'MITM-ArpSpoofing', 23: 'Mirai-greeth_flood', 24: 'Mirai-greip_flood', 25: 'Mirai-udpplain', 26: 'Recon-HostDiscovery', 27: 'Recon-OSScan', 28: 'Recon-PingSweep', 29: 'Recon-PortScan', 30: 'SqlInjection', 31: 'Uploading_Attack', 32: 'VulnerabilityScan', 33: 'XSS'}
After encoding:
First instance of Backdoor_Malware (code 0):
flow_duration      9.265858e-01
Header_Length      9.335040e+04
Protocol Type      5.500000e+00
Duration           5.120000e+01
Rate

## Scaling Numeric Features

In [4]:
# feature scaling
scaler = StandardScaler()

for data_set in tqdm(data_sets):
    scaler.fit(pd.read_csv(DATASET_DIRECTORY + data_set)[num_cols])
    
# Scale the features in the dataframe
full_data[num_cols] = scaler.transform(full_data[num_cols])

# Display the first few entries to verify the changes
print(full_data)

# prep the data to be inputted into model
data = full_data

100%|██████████| 1/1 [00:00<00:00,  1.18it/s]

        flow_duration  Header_Length  Protocol Type  Duration      Rate  \
33308       -0.006349      -0.168361           6.00 -0.167903 -0.092665   
187239      -0.021994      -0.144313          17.00 -0.167903  0.100247   
11907       -0.022256      -0.168477           6.00 -0.167903 -0.092329   
63608       -0.022256      -0.168477           6.00 -0.167903 -0.092667   
107281       0.047345      -0.168167           6.00 -0.044655 -0.092669   
...               ...            ...            ...       ...       ...   
109259      -0.022169      -0.151982          17.00 -0.167903  0.249658   
50057       -0.022256      -0.168477           6.00 -0.167903 -0.092655   
5192        -0.022256      -0.168477           6.00 -0.167903 -0.092281   
208780      -0.022252      -0.168475           6.11 -0.182069 -0.092288   
128037      -0.021646      -0.125428          17.00 -0.167903 -0.033609   

           Srate  Drate  fin_flag_number  syn_flag_number  rst_flag_number  \
33308  -0.092665    0

## Preprocessed DataFrame

In [5]:
full_data.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
33308,-0.006349,-0.168361,6.0,-0.167903,-0.092665,-0.092665,0.0,0.0,1.0,0.0,...,-0.206322,-0.296223,0.010889,0.00294,-0.320215,-0.206144,-0.090697,-0.415473,0.002821,12
187239,-0.021994,-0.144313,17.0,-0.167903,0.100247,0.100247,0.0,0.0,0.0,0.0,...,-0.206322,-0.312759,-0.003031,0.00294,-0.365427,-0.206144,-0.090697,-0.415473,0.002821,14
11907,-0.022256,-0.168477,6.0,-0.167903,-0.092329,-0.092329,0.0,0.0,0.0,0.0,...,-0.206322,-0.296223,-0.006077,0.00294,-0.320215,-0.206144,-0.090697,-0.415473,0.002821,13
63608,-0.022256,-0.168477,6.0,-0.167903,-0.092667,-0.092667,0.0,0.0,0.0,0.0,...,-0.206322,-0.296223,-0.006271,0.00294,-0.320215,-0.206144,-0.090697,-0.415473,0.002821,13
107281,0.047345,-0.168167,6.0,-0.044655,-0.092669,-0.092669,0.0,0.0,0.0,0.0,...,-0.189912,-0.292626,-0.013613,0.00294,-0.311952,-0.189708,-0.090524,0.098145,0.002821,20


# Sampling (SMOTE Over-Sampling)

In [6]:
min_class_size = full_data['label'].value_counts().min()

# Apply random over-sampling
smote_os = SMOTE(k_neighbors=4, random_state=42)

X = full_data.drop('label', axis=1)
y = full_data['label']
X_resampled, y_res = smote_os.fit_resample(X, y)

# Combine the resampled features and labels back into a single DataFrame
full_data_resampled = pd.DataFrame(X_resampled, columns=X.columns)
full_data_resampled['label'] = y_res

print("Resampled Data (SCALED):")
for label, code in class_codes.items():
    # Print the first instance of each class
    print(f"First instance of {label} (code {code}):")
    print(full_data_resampled[full_data_resampled['label'] == code].iloc[0])

full_data_resampled.head()

Resampled Data (SCALED):
First instance of Backdoor_Malware (code 0):
flow_duration     -0.018820
Header_Length      0.032414
Protocol Type      5.500000
Duration          -1.074550
Rate              -0.091767
Srate             -0.091767
Drate              0.000000
fin_flag_number    0.000000
syn_flag_number    0.000000
rst_flag_number    0.000000
psh_flag_number    0.000000
ack_flag_number    1.000000
ece_flag_number    0.000000
cwr_flag_number    0.000000
ack_count         -0.316729
syn_count          2.229601
fin_count         -0.290548
urg_count          0.013052
rst_count          0.118630
HTTP               0.000000
HTTPS              1.000000
DNS                0.000000
Telnet             0.000000
SMTP               0.000000
SSH                0.000000
IRC                0.000000
TCP                1.000000
UDP                0.000000
DHCP               0.000000
ARP                0.000000
ICMP               0.000000
IPv                1.000000
LLC                1.000000
Tot su

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,-0.006349,-0.168361,6.0,-0.167903,-0.092665,-0.092665,0.0,0.0,1.0,0.0,...,-0.206322,-0.296223,0.010889,0.00294,-0.320215,-0.206144,-0.090697,-0.415473,0.002821,12
1,-0.021994,-0.144313,17.0,-0.167903,0.100247,0.100247,0.0,0.0,0.0,0.0,...,-0.206322,-0.312759,-0.003031,0.00294,-0.365427,-0.206144,-0.090697,-0.415473,0.002821,14
2,-0.022256,-0.168477,6.0,-0.167903,-0.092329,-0.092329,0.0,0.0,0.0,0.0,...,-0.206322,-0.296223,-0.006077,0.00294,-0.320215,-0.206144,-0.090697,-0.415473,0.002821,13
3,-0.022256,-0.168477,6.0,-0.167903,-0.092667,-0.092667,0.0,0.0,0.0,0.0,...,-0.206322,-0.296223,-0.006271,0.00294,-0.320215,-0.206144,-0.090697,-0.415473,0.002821,13
4,0.047345,-0.168167,6.0,-0.044655,-0.092669,-0.092669,0.0,0.0,0.0,0.0,...,-0.189912,-0.292626,-0.013613,0.00294,-0.311952,-0.189708,-0.090524,0.098145,0.002821,20


# Post-Processing
## Inverse-Scaling

In [7]:
full_data_resampled[num_cols] = scaler.inverse_transform(full_data_resampled[num_cols], copy=None)

print("Resampled Data (UNSCALED):")
for label, code in class_codes.items():
    # Print the first instance of each class
    print(f"First instance of {label} (code {code}):")
    print(full_data_resampled[full_data_resampled['label'] == code].iloc[0])

print("Number of negative values in DataFrame: ")
print(sum(n < 0 for n in full_data_resampled.values.flatten()))

full_data_resampled.head()

Resampled Data (UNSCALED):
First instance of Backdoor_Malware (code 0):
flow_duration      9.265858e-01
Header_Length      9.335040e+04
Protocol Type      5.500000e+00
Duration           5.120000e+01
Rate               8.575327e+01
Srate              8.575327e+01
Drate              0.000000e+00
fin_flag_number    0.000000e+00
syn_flag_number    0.000000e+00
rst_flag_number    0.000000e+00
psh_flag_number    0.000000e+00
ack_flag_number    1.000000e+00
ece_flag_number    0.000000e+00
cwr_flag_number    0.000000e+00
ack_count          0.000000e+00
syn_count          1.800000e+00
fin_count          0.000000e+00
urg_count          7.200000e+00
rst_count          7.830000e+01
HTTP               0.000000e+00
HTTPS              1.000000e+00
DNS                0.000000e+00
Telnet             0.000000e+00
SMTP               0.000000e+00
SSH                0.000000e+00
IRC                0.000000e+00
TCP                1.000000e+00
UDP                0.000000e+00
DHCP               0.000000e+00


,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,4.288836,108.0,6.0,64.00,0.532663,0.532663,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336126e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,12
1,0.070704,11276.0,17.0,64.00,18298.327225,18298.327225,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.312357e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,14
2,0.000000,54.0,6.0,64.00,32.403336,32.403336,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.307157e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,13
3,0.000000,54.0,6.0,64.00,0.399501,0.399501,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.306827e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,13
4,18.764876,198.0,6.0,65.74,0.146889,0.146889,0.0,0.0,0.0,0.0,...,2.696344,54.87,8.294290e+07,9.5,10.464001,3.819791,61.574028,0.12,141.55,20


## Synthetic Data Analysis

In [8]:
full_data_resampled.describe()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
count,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,...,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06,1.139986e+06
mean,1.041765e+02,2.169715e+05,1.043356e+01,7.847432e+01,4.892831e+03,4.892831e+03,1.775804e-07,2.964556e-02,1.102261e-01,4.902720e-02,...,1.750225e+02,3.266268e+02,8.794445e+07,9.717141e+00,2.116006e+01,2.474353e+02,1.404028e+05,5.587166e-01,1.471535e+02,1.650000e+01
std,7.307171e+02,7.889024e+05,9.771454e+00,2.800244e+01,6.281619e+04,6.281619e+04,4.906756e-05,1.693637e-01,3.115322e-01,2.145552e-01,...,2.896947e+02,4.058108e+02,5.316590e+07,2.548059e+00,1.306832e+01,4.098340e+02,5.125615e+05,4.374696e-01,6.562268e+01,9.810713e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,4.200000e+01,0.000000e+00,1.000000e+00,9.165151e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,6.666375e-03,5.886000e+01,6.000000e+00,6.400000e+01,4.789924e+00,4.789924e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,5.412000e+01,8.299973e+07,9.500000e+00,1.039943e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.415500e+02,8.000000e+00
50%,2.451058e+00,4.719151e+03,6.358265e+00,6.464206e+01,2.421524e+01,2.421524e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.820600e+01,1.223556e+02,8.333087e+07,9.500000e+00,1.539078e+01,6.808737e+01,3.681009e+03,8.399309e-01,1.415500e+02,1.650000e+01
75%,4.845046e+01,3.841259e+04,1.127592e+01,8.577678e+01,1.063859e+02,1.063859e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.270818e+02,5.540000e+02,8.376932e+07,9.500000e+00,3.307644e+01,3.209719e+02,9.932996e+04,9.500000e-01,1.415500e+02,2.500000e+01
max,6.573530e+04,9.815555e+06,4.700000e+01,2.550000e+02,6.291456e+06,6.291456e+06,2.202362e-02,1.000000e+00,1.000000e+00,1.000000e+00,...,6.225799e+03,5.091600e+03,1.676394e+08,1.350000e+01,1.120220e+02,8.804609e+03,4.432982e+07,1.000000e+00,2.446000e+02,3.300000e+01


In [ ]:
from ydata_profiling import ProfileReport

original_report = ProfileReport(full_data, title='Original Data')
original_report.to_file("original_report.html")

resampled_report = ProfileReport(full_data_resampled, title='Resampled Data')
resampled_report.to_file("resampled_report.html")

comparison_report = original_report.compare(resampled_report)
comparison_report.to_file('original_vs_resampled.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]